# Glider Toolbox Demo Notebook

### Using a Pre-existing config file for a complete pipeline run

Note: Currently these imports are local references. In the future the toolbox will be an installable package through pip.

In [1]:
# add toolbox to src (THIS WILL NOT BE REQUIRED ONCE DEPLOYED)
import sys
from pathlib import Path
src_path = Path("../../src").resolve()

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [ ]:
from toolbox.pipeline import Pipeline


pipeline = Pipeline(   
    "../configs/example_config_doombar.yaml")
pipeline.run()

### Building the pipeline one step at a time

In [ ]:
pipeline2 = Pipeline()

# add load step
pipeline2.add_step(
    "Load OG1",
    parameters={
        "file_path": "../../examples/data/OG1/Churchill_647_R.nc",  # Path to the input NetCDF file
        "add_meta": False, # Add metadata to the data
        "add_depth": True, # Add depth information to the data
        "add_elapsed_time": False,
        "lat_label": "DEPLOYMENT_LATITUDE"},
    diagnostics=False,
    run_immediately=True )# can run immediately upon creation

# add export step
pipeline2.add_step(
    "Data Export",
    parameters={
        "export_format": "netcdf",  # Define the export format (e.g., OG1, CSV, etc.)
        "output_path": "../../examples/data/OG1/exported_Churchill_647_R.nc"},
    diagnostics=False,
    run_immediately=False )# or run at a future time
# run last step
pipeline2.run_last_step()

In [ ]:
conf = pipeline2.export_config("../../examples/data/OG1/Churchill_647_R_CONFIG.yaml")  # Export the pipeline configuration to a YAML file
conf

# Running Multiple Pipelines

```python

In [2]:
### Running multiple pipelines using the PipelineManager class
from toolbox.pipeline import PipelineManager
mngr = PipelineManager()
mngr.load_mission_control("../configs/MissionControl.yaml")  # Load the mission control configuration
# This is currently set up to import data, run profiles, and export data for Doombar and Churchill sources.
mngr.run_all()  # Run all pipelines defined in the mission control configuration

[Discovery] Scanning for step modules in /home/adamwa/Ocean Informatics/Projects/toolbox/src/toolbox/steps/custom
[Discovery] Importing step module: toolbox.steps.custom.export
[Discovery] Importing step module: toolbox.steps.custom.load_data
[Discovery] Importing step module: toolbox.steps.custom.calibration
[Discovery] Importing step module: toolbox.steps.custom.find_profiles
[Discovery] Importing step module: toolbox.steps.custom.variables.salinity
[Discovery] Registered step: Data Export
[Discovery] Registered step: Load OG1
[Discovery] Registered step: Find Profiles
[Discovery] Registered step: QC: Salinity
[Discovery] Registered step: ADJ: Salinity
Step 'Load OG1' added successfully!
Step 'Find Profiles' added successfully!
Step 'Data Export' added successfully!
Step 'Load OG1' added successfully!
Step 'Find Profiles' added successfully!
Step 'Data Export' added successfully!
####################
Running pipeline: Churchill
Executing: Load OG1
[Load OG1] Params: {'file_path': '/h

TypeError: 'NoneType' object cannot be interpreted as an integer